In [1]:
import os 
import pandas as pd
from nilearn import datasets
from bwas_wrapper.wrapper import wrapper

Let's start by downloading a few subjects from the ADHD200 dataset.

In [2]:
n_subjects = 30
adhd_dataset = datasets.fetch_adhd(n_subjects)
epi_filename = adhd_dataset.func
# Also load a probabilistic grey matter segmentation
mni = datasets.fetch_icbm152_2009()

/home/pbellec/env/simonsvip/lib/python3.7/site-packages/nilearn/datasets/func.py:516: VisibleDeprecationWarning: Reading unicode strings without specifying the encoding argument is deprecated. Set the encoding, use None for the system default.
  dtype=None)


Let's load the phenotypic information in a pandas DataFrame and check what variables there are:

In [3]:
data = pd.DataFrame(adhd_dataset.phenotypic)
data.columns

Index(['f0', 'Subject', 'RestScan', 'MeanFD', 'NumFD_greater_than_020',
       'rootMeanSquareFD', 'FDquartiletop14thFD', 'PercentFD_greater_than_020',
       'MeanDVARS', 'MeanFD_Jenkinson', 'site', 'sibling_id', 'data_set',
       'age', 'sex', 'handedness', 'full_2_iq', 'full_4_iq', 'viq', 'piq',
       'iq_measure', 'tdc', 'adhd', 'adhd_inattentive', 'adhd_combined',
       'adhd_subthreshold', 'diagnosis_using_cdis', 'notes', 'sess_1_anat_2',
       'oppositional', 'cog_inatt', 'hyperac', 'anxious_shy', 'perfectionism',
       'social_problems', 'psychosomatic', 'conn_adhd', 'restless_impulsive',
       'emot_lability', 'conn_gi_tot', 'dsm_iv_inatt', 'dsm_iv_h_i',
       'dsm_iv_tot', 'study', 'sess_1_rest_1', 'sess_1_rest_1_eyes',
       'sess_1_rest_2', 'sess_1_rest_2_eyes', 'sess_1_rest_3',
       'sess_1_rest_3_eyes', 'sess_1_rest_4', 'sess_1_rest_4_eyes',
       'sess_1_rest_5', 'sess_1_rest_5_eyes', 'sess_1_rest_6',
       'sess_1_rest_6_eyes', 'sess_1_anat_1', 'sess_1_which

We need to select the variables to include in our analyis, and also have a column with the subject IDs:

In [4]:
model = data[['Subject', 'adhd', 'MeanFD', 'age']]

let's add sex as well, but this variable is categorical. We need to convert it into a dummy variable first.

In [5]:
sex = pd.get_dummies(data['sex'], prefix='sex', drop_first=True) # drop_first is applied because the model will include an intercept
model = pd.concat([model, sex], axis=1)

we can select subjects too. For example, let's filter subjects with excessive motion.

In [6]:
model = model[model['MeanFD']<0.1]

Let's add an intercept to the regression model:

In [7]:
model.insert(0, column='Intercept', value=1)

Finally, we need to add a dictionary with the file of the preprocessed data for each subject. The IDs of the subject need to correspond to the subject IDs in the data frame. 

In [8]:
# By checking `adhd_dataset.func`, we can easily pick up the naming patterns of the functional files
# '/home/pbellec/nilearn_data/adhd/data/0010042/0010042_rest_tshift_RPI_voreg_mni.nii.gz'
path_data = os.path.join( os.getenv('HOME'), 'nilearn_data', 'adhd')
files = {}
for subject in model['Subject']:
    file = f"{ subject }_rest_tshift_RPI_voreg_mni.nii.gz"
    files[subject] = os.path.join(path_data, file)
files

{10042: '/home/pbellec/nilearn_data/adhd/10042_rest_tshift_RPI_voreg_mni.nii.gz',
 10064: '/home/pbellec/nilearn_data/adhd/10064_rest_tshift_RPI_voreg_mni.nii.gz',
 10128: '/home/pbellec/nilearn_data/adhd/10128_rest_tshift_RPI_voreg_mni.nii.gz',
 21019: '/home/pbellec/nilearn_data/adhd/21019_rest_tshift_RPI_voreg_mni.nii.gz',
 23008: '/home/pbellec/nilearn_data/adhd/23008_rest_tshift_RPI_voreg_mni.nii.gz',
 23012: '/home/pbellec/nilearn_data/adhd/23012_rest_tshift_RPI_voreg_mni.nii.gz',
 27018: '/home/pbellec/nilearn_data/adhd/27018_rest_tshift_RPI_voreg_mni.nii.gz',
 27034: '/home/pbellec/nilearn_data/adhd/27034_rest_tshift_RPI_voreg_mni.nii.gz',
 27037: '/home/pbellec/nilearn_data/adhd/27037_rest_tshift_RPI_voreg_mni.nii.gz',
 1019436: '/home/pbellec/nilearn_data/adhd/1019436_rest_tshift_RPI_voreg_mni.nii.gz',
 1206380: '/home/pbellec/nilearn_data/adhd/1206380_rest_tshift_RPI_voreg_mni.nii.gz',
 1418396: '/home/pbellec/nilearn_data/adhd/1418396_rest_tshift_RPI_voreg_mni.nii.gz',
 151

In [9]:
wrapper(model, files, output_folder='/tmp/test/', subject_id="Subject", interest="adhd")

Creating sym links in the following path: /tmp/tmpp7zcc2xp


NameError: name 'BWAS_run_full_analysis' is not defined